# 众所周知我要干什么
 目前做完了第一步

 下载并且解压数据

# 数据读入

In [1]:
import torch
import cv2
import os
import glob
from torch.utils.data import Dataset
import random

class ISBI_Loader(Dataset):
    def __init__(self, data_path):
        # 初始化函数，读取所有data_path下的图片‘
        print(data_path)
        if(data_path == ''):
            return 
        self.data_path = data_path
        _tester_watch =  data_path + '/input/*.png'
        # self.imgs_path = glob.glob(os.path.join(data_path, 'input/*.png'))
        # self.imgs_path = glob.glob(_tester_watch)
#         self.imgs_path = os.listdir('/kaggle/working'+'/input')
        self.imgs_path = os.listdir(data_path+'/input')
        for i in range(len(self.imgs_path)):
            self.imgs_path[i] = data_path + '/input/' + self.imgs_path[i]

    def augment(self, image, flipCode):
        # 使用cv2.flip进行数据增强，filpCode为1水平翻转，0垂直翻转，-1水平+垂直翻转
        # 我们是彩色图像，不用管
        # flip = cv2.flip(image, flipCode)
        return image
        # return flip

    def __getitem__(self, index):
        # 根据index读取图片
        image_path = self.imgs_path[index]
        # 根据image_path生成label_path
        label_path = image_path
        tempsplit = label_path.split('/')
        tempsplit[len(tempsplit)-2]='output'
        label_path = tempsplit[0]
        for i in range(len(tempsplit)-1):
            label_path=label_path+'/'+tempsplit[i+1]
#         label_path = image_path.replace('input', 'output')
        # 读取训练图片和标签图片
#         print(label_path)
        image = cv2.imread(image_path)
        label = cv2.imread(label_path)
        # 将数据转为单通道的图片
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        label = cv2.cvtColor(label, cv2.COLOR_BGR2GRAY)
        image = image.reshape(1, image.shape[0], image.shape[1])
        label = label.reshape(1, label.shape[0], label.shape[1])
        # 处理标签，将像素值为255的改为1
        if label.max() > 1:
            label = label / 255
        # 随机进行数据增强，为2时不做处理
        flipCode = random.choice([-1, 0, 1, 2])
        if flipCode != 2:
            image = self.augment(image, flipCode)
            label = self.augment(label, flipCode)
        return image, label

    def __len__(self):
        # 返回训练集大小
        return len(self.imgs_path)


# if __name__ == "__main__":
#     isbi_dataset = ISBI_Loader("/kaggle/working/littletrain")
#     print("数据个数：", len(isbi_dataset))
#     train_loader = torch.utils.data.DataLoader(dataset=isbi_dataset,
#                                                batch_size=32,
#                                                shuffle=True)
#     for image, label in train_loader:
#         print(image.shape)

# Unet模块

In [2]:
""" Parts of the U-Net model """
"""https://github.com/milesial/Pytorch-UNet/blob/master/unet/unet_parts.py"""

import torch
import torch.nn as nn
import torch.nn.functional as F


class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_channels // 2, in_channels // 2, kernel_size=2, stride=2)

        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = torch.tensor([x2.size()[2] - x1.size()[2]])
        diffX = torch.tensor([x2.size()[3] - x1.size()[3]])

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])

        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

In [3]:
""" Full assembly of the parts to form the complete network """
"""Refer https://github.com/milesial/Pytorch-UNet/blob/master/unet/unet_model.py"""

import torch.nn.functional as F

# from .unet_parts import *


class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 512)
        self.up1 = Up(1024, 256, bilinear)
        self.up2 = Up(512, 128, bilinear)
        self.up3 = Up(256, 64, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

if __name__ == '__main__':
    net = UNet(n_channels=3, n_classes=1)
    print(net)

UNet(
  (inc): DoubleConv(
    (double_conv): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (down1): Down(
    (maxpool_conv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConv(
        (double_conv): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (4): BatchNorm2d(128, eps=1e-05, moment

# 开始训练

In [4]:
# from model.unet_model import UNet
# from utils.dataset import ISBI_Loader
from torch import optim
import torch.nn as nn
import torch

filepath = '/kaggle/input/little' #填写当前目录

def train_net(net, device, data_path, epochs=40, batch_size=1, lr=0.00001):
    # 加载训练集
    train_path = data_path +'/train'
    test_path = data_path+ '/test'
    isbi_train_dataset = ISBI_Loader(train_path)
    train_loader = torch.utils.data.DataLoader(dataset=isbi_train_dataset,
                                               batch_size=batch_size,
                                               shuffle=True)
    isbi_test_dataset = ISBI_Loader(test_path)
    test_loader = torch.utils.data.DataLoader(dataset=isbi_test_dataset,
                                               batch_size=batch_size,
                                               shuffle=True)
    # 定义RMSprop算法
    optimizer = optim.RMSprop(net.parameters(), lr=lr, weight_decay=1e-8, momentum=0.9)
    # 定义Loss算法
    criterion = nn.BCEWithLogitsLoss()
    # best_loss统计，初始化为正无穷
    best_loss = float('inf')
    # 训练epochs次
    for epoch in range(epochs):
        # 训练模式
        net.train()
        # 按照batch_size开始训练
        losss = 0
        for image, label in train_loader:
            optimizer.zero_grad()
            # 将数据拷贝到device中
            image = image.to(device=device, dtype=torch.float32)
            label = label.to(device=device, dtype=torch.float32)
            # 使用网络参数，输出预测结果
            pred = net(image)
            # 计算loss
            loss  = criterion(pred, label)
            losss = losss + loss.item()
            
            # 更新参数
            loss.backward()
            optimizer.step()
        print('train : Loss/train', losss/len(train_loader))
        net.eval()
        losss = 0
        for image, label in test_loader:
            optimizer.zero_grad()
            # 将数据拷贝到device中
            image = image.to(device=device, dtype=torch.float32)
            label = label.to(device=device, dtype=torch.float32)
            # 使用网络参数，输出预测结果
            pred = net(image)
            # 计算loss
            loss = criterion(pred, label)
            losss = losss + loss.item()
            # 保存loss值最小的网络参数
        print('test : Loss/train',losss/len(test_loader))
        if loss < best_loss:
            best_loss = loss
            torch.save(net.state_dict(), 'best_model.pth')

if __name__ == "__main__":
    # 选择设备，有cuda用cuda，没有就用cpu
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # device = 'cpu'
    # 加载网络，图片单通道1，分类为1。
    torch.cuda.empty_cache() # 显然我们有cuda
    net = UNet(n_channels=1, n_classes=1)
    # 将网络拷贝到deivce中
    net.to(device=device)
    # 指定训练集地址，开始训练
    data_path = filepath + '/littletrain'
    train_net(net, device, data_path)

/kaggle/input/little/littletrain/train
/kaggle/input/little/littletrain/test
train : Loss/train 0.622371240456899
test : Loss/train 0.8811939239501954
train : Loss/train 0.5725077271461487
test : Loss/train 0.5965171098709107
train : Loss/train 0.5662465214729309
test : Loss/train 0.5652386307716369
train : Loss/train 0.5635722895463308
test : Loss/train 0.5605841636657715
train : Loss/train 0.5617866416772207
test : Loss/train 0.5605167388916016
train : Loss/train 0.5620243589083354
test : Loss/train 0.5853234052658081
train : Loss/train 0.5617138107617696
test : Loss/train 0.5604898571968079
train : Loss/train 0.5638190388679505
test : Loss/train 0.5630278825759888
train : Loss/train 0.5622169375419617
test : Loss/train 0.5601600408554077
train : Loss/train 0.5601984262466431
test : Loss/train 0.5596546769142151
train : Loss/train 0.5674891968568166
test : Loss/train 0.5628050208091736
train : Loss/train 0.5638785084088643
test : Loss/train 0.5617167830467225
train : Loss/train 0.560

# 完成训练之后评估